# 🔍 Query Records by IDs

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
# Create Spark Session
spark = SparkSession.builder \
    .appName("QueryByIds") \
    .master("local[*]") \
    .config("spark.es.nodes", "elasticsearch") \
    .config("spark.es.port", "9200") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:8.14.3") \
    .getOrCreate()

print(f"Application ID: {spark.sparkContext.applicationId}")

Application ID: local-1757837619139


In [3]:
# List of IDs to query (you can modify this list)
query_ids = [24410114, 24410100, 24410109, 24410092, 24410040]
print(f"Searching for IDs: {query_ids}")

Searching for IDs: [24410114, 24410100, 24410109, 24410092, 24410040]


In [4]:
# Read data from Elasticsearch
df = spark.read \
    .format("org.elasticsearch.spark.sql") \
    .option("es.nodes", "elasticsearch") \
    .option("es.port", "9200") \
    .option("es.resource", "2_people_data_2k") \
    .load()

print(f"Total records in index: {df.count()}")

Total records in index: 2006


In [5]:
# Filter by IDs
filtered_df = df.filter(col("id").isin(query_ids))
found_count = filtered_df.count()

print(f"Found {found_count} matching records:")

if found_count > 0:
    # Show all matching records
    filtered_df.orderBy("id").show()

Found 5 matching records:
+----------+--------+---+-----+--------+----+--------+----+-------+--------------------+
|@timestamp|@version|age|event|filename|host|      id| log|message|                name|
+----------+--------+---+-----+--------+----+--------+----+-------+--------------------+
|      NULL|    NULL| 22| NULL|    NULL|NULL|24410040|NULL|   NULL| Ha Huy Hung updated|
|      NULL|    NULL| 35| NULL|    NULL|NULL|24410092|NULL|   NULL|Huynh Duy Quoc up...|
|      NULL|    NULL| 30| NULL|    NULL|NULL|24410100|NULL|   NULL|Nguyen Phuong Tan...|
|      NULL|    NULL| 28| NULL|    NULL|NULL|24410109|NULL|   NULL|Nguyen Thi Thu Th...|
|      NULL|    NULL| 30| NULL|    NULL|NULL|24410114|NULL|   NULL|Tran Tireu Thuan ...|
+----------+--------+---+-----+--------+----+--------+----+-------+--------------------+



In [7]:
if found_count > 0:
    # Show summary statistics
    print("Summary statistics for found records:")
    age_stats = filtered_df.agg(min('age'), max('age'), avg('age')).collect()[0]
    print(f"  - Age range: {age_stats[0]} to {age_stats[1]}")
    print(f"  - Average age: {age_stats[2]:.1f}")
    
    # Show which IDs were found vs not found
    found_ids = [row['id'] for row in filtered_df.select('id').collect()]
    not_found_ids = [id for id in query_ids if id not in found_ids]
    
    print(f"Found IDs: {sorted(found_ids)}")
    if not_found_ids:
        print(f"Not found IDs: {sorted(not_found_ids)}")
else:
    print("No records found with the specified IDs.")

Summary statistics for found records:
  - Age range: 22 to 35
  - Average age: 29.0
Found IDs: [24410040, 24410092, 24410100, 24410109, 24410114]


In [8]:
# Stop Spark session
spark.stop()
print("Spark session stopped.")

Spark session stopped.
